In [1]:
# generate links / json states programatically by cell type or anything, probably hack-y but it works
from caveclient import CAVEclient
import datetime
import pandas as pd
import numpy as np
import json

client = CAVEclient()
datastack_name = 'fanc_production_mar2021'
client = CAVEclient(datastack_name)
client.materialize.version = 604
timestamp = datetime.datetime.fromtimestamp(1685002201.254304)

In [2]:
# toggle

ngl_base = 'https://neuromancer-seung-import.appspot.com/' 


In [56]:
def make_json2(seg_ids, hidden_ids):
    state = {
    "layers": [
      {
        "source": "precomputed://gs://lee-lab_female-adult-nerve-cord/alignmentV4/em/rechunked",
        "type": "image",
        "blend": "default",
        "shaderControls": {},
        "name": "FANC EM"
      },
      {
        "type": "segmentation",
        "mesh": "precomputed://gs://lee-lab_female-adult-nerve-cord/meshes/FANC/FANC_neurons/meshes",
        "selectedAlpha": 0.4,
        "colorSeed": 3788587020,
        "segments": seg_ids,
        "hiddenSegments": hidden_ids,
        "skeletonRendering": {
          "mode2d": "lines_and_points",
          "mode3d": "lines"
        },
        "name": "published FANC neurons"
      },
      {
        "type": "segmentation",
        "mesh": "precomputed://gs://lee-lab_female-adult-nerve-cord/alignmentV4/volume_meshes/meshes",
        "objectAlpha": 0.1,
        "hideSegmentZero": False,
        "ignoreSegmentInteractions": True,
        "segmentColors": {
          "1": "#bfbfbf",
          "2": "#d343d6"
        },
        "segments": [
          "1",
          "2"
        ],
        "skeletonRendering": {
          "mode2d": "lines_and_points",
          "mode3d": "lines"
        },
        "name": "region outlines"
      }
    ],
    "navigation": {
      "pose": {
        "position": {
          "voxelSize": [
            4.300000190734863,
            4.300000190734863,
            45
          ],
          "voxelCoordinates": [38734, 148426, 2200
          ]
        }
      },
      "zoomFactor": 12
    },
    "showAxisLines": False,
    "showDefaultAnnotations": False,
    "perspectiveZoom": 6062.41070084089,
    "showSlices": False,
    "gpuMemoryLimit": 4000000000,
    "systemMemoryLimit": 4000000000,
    "concurrentDownloads": 64,
    "jsonStateServer": "https://global.daf-apis.com/nglstate/api/v1/post",
    "selectedLayer": {
      "layer": "published FANC neurons",
      "visible": True
    },
    "layout": "3d"
  }

    return state

In [35]:
leg_df = pd.read_pickle('dfs_pre_to_mn/pre_to_mn_df_matched_typed_with_nt_v604_20230525.pkl')
leg_mns = leg_df.columns.get_level_values('segID').to_list()

In [44]:
# need a programattic way to do this
module_mns = [648518346496059082,648518346494196727,648518346496932836,648518346484809885,648518346493836024,648518346486906595,648518346482926501,648518346489731855,648518346494269831,648518346499176513]

In [53]:
seg_ids = module_mns
hidden_ids = leg_mns
json_title = 'mn_tibia_flex_A'

formatted_ids = [str(x) for x in seg_ids]
formatted_hidden_ids = [str(x) for x in hidden_ids]

state = make_json(seg_ids = formatted_ids,
                hidden_ids = formatted_hidden_ids)

In [ ]:
# save a json
with open('jsons/{}.json'.format(json_title), "w") as outfile:
    json.dump(state, outfile)

In [55]:
# make a link
new_id = client.state.upload_state_json(state)
client.state.build_neuroglancer_url(new_id, ngl_base)

'https://neuromancer-seung-import.appspot.com/?json_url=https://global.daf-apis.com/nglstate/api/v1/5634631871234048'

In [ ]:
# trouble shooting and old code

In [54]:
state

{'layers': [{'source': 'precomputed://gs://lee-lab_female-adult-nerve-cord/alignmentV4/em/rechunked',
   'type': 'image',
   'blend': 'default',
   'shaderControls': {},
   'name': 'FANC EM'},
  {'type': 'segmentation',
   'mesh': 'precomputed://gs://lee-lab_female-adult-nerve-cord/meshes/FANC/FANC_neurons/meshes',
   'selectedAlpha': 0.4,
   'colorSeed': 3788587020,
   'segments': ['648518346496059082',
    '648518346494196727',
    '648518346496932836',
    '648518346484809885',
    '648518346493836024',
    '648518346486906595',
    '648518346482926501',
    '648518346489731855',
    '648518346494269831',
    '648518346499176513'],
   'hiddenSegments': ['648518346487756866',
    '648518346491659326',
    '648518346517437482',
    '648518346517827432',
    '648518346479325585',
    '648518346487504531',
    '648518346486895331',
    '648518346479077839',
    '648518346489076201',
    '648518346490902957',
    '648518346489802071',
    '648518346478992848',
    '648518346488854257',
 

In [4]:
# import dataframe
wing_df = pd.read_pickle('preMN_to_MN_wing_v604_multi_df.pkl')

In [30]:
# generate a link based on a cell type
# only for local and intersegmental preMNs

idx = pd.IndexSlice
df = wing_df.loc[idx[:,:,:,['local','local_intersegmental'],:]].reset_index() # group by cell type

cell_types = df.cell_type.unique()

for cell_type in cell_types[1:2]:
    cell_type_label = str(cell_type)
    seg_ids = df[df.cell_type.isin([cell_type_label])].pre_pt_root_id.to_list()
    formatted_ids = [str(x) for x in seg_ids]
    
    state = make_json(name = cell_type_label,
                   seg_ids = formatted_ids)
    new_id = client.state.upload_state_json(state)
    client.state.build_neuroglancer_url(new_id, ngl_base)

    # save json 
    # with open('jsons/{}.json'.format(cell_type_label), "w") as outfile:
    #     json.dump(state, outfile)



In [31]:
client.state.build_neuroglancer_url(new_id, ngl_base)


'https://neuromancer-seung-import.appspot.com/?json_url=https://global.daf-apis.com/nglstate/api/v1/5047254895296512'

In [29]:
# generate a link based on preferred module

idx = pd.IndexSlice
df = wing_df.reset_index() # group by cell type

modules = df.preferred_module.unique()

for module in modules:
    module_label = str(module)
    seg_ids = df[df.preferred_module.isin([module_label])].pre_pt_root_id.to_list()
    formatted_ids = [str(x) for x in seg_ids]
    
    state = make_json(name = module_label,
                   seg_ids = formatted_ids)
    # new_id = client.state.upload_state_json(state)
    # client.state.build_neuroglancer_url(new_id, ngl_base)

    # save json 
    with open('jsons/{}.json'.format(module_label), "w") as outfile:
        json.dump(state, outfile)


In [11]:
# jsons per cell class / per preferred module 

idx = pd.IndexSlice
df = wing_df.reset_index() # group by cell type

modules = df.preferred_module.unique()
cell_classes = df.cell_class.unique()

for module in modules:
    module_label = str(module)
    module_df = df[df.preferred_module.isin([module_label])]

    for cell in cell_classes:
        cell_class_label = str(cell)
        
        seg_ids = module_df[module_df.cell_class.isin([cell_class_label])].pre_pt_root_id.to_list()
        formatted_ids = [str(x) for x in seg_ids]
    
        statename = 'preferred_module_{}_{}'.format(module_label[:-4],cell_class_label) 
        state = make_json(name = statename,
                          seg_ids = formatted_ids)
        # new_id = client.state.upload_state_json(state)
        # client.state.build_neuroglancer_url(new_id, ngl_base)

    # save json
        with open('jsons/{}.json'.format(statename), "w") as outfile:
            json.dump(state, outfile)



In [25]:
# module jsons 
mn_df = pd.read_pickle('wingMN_properties_v604.pkl')

# modules
DLM_mod = ['DLM_1','DLM_2','DLM_3','DLM_4','DLM_5']
DVM_mod = ['DVM_1a','DVM_1b','DVM_1c','DVM_2a','DVM_2b','DVM_3a','DVM_3b']
tension_mod = ['PS1','PSn_u','PS2','dtpmn_u','tp2']
steerA_mod = ['tpn','iii1','iii3']
steerB_mod = ['i2_u','i1','b3_u']
steerC_mod = ['b2','b1']
hg2_mod = ['hg2']
steerD_mod = ['hg1_u','hg3','hg4','iii4a_u']

mn_df.loc[mn_df.MN_label.isin(DLM_mod), 'motor_module'] = 'DLM_module'
mn_df.loc[mn_df.MN_label.isin(DVM_mod), 'motor_module'] = 'DVM_module'
mn_df.loc[mn_df.MN_label.isin(tension_mod), 'motor_module'] = 'tension_module'
mn_df.loc[mn_df.MN_label.isin(steerA_mod), 'motor_module'] = 'steerA_module'
mn_df.loc[mn_df.MN_label.isin(steerB_mod), 'motor_module'] = 'steerB_module'
mn_df.loc[mn_df.MN_label.isin(steerC_mod), 'motor_module'] = 'steerC_module'
mn_df.loc[mn_df.MN_label.isin(hg2_mod), 'motor_module'] = 'hg2_module'
mn_df.loc[mn_df.MN_label.isin(steerD_mod), 'motor_module'] = 'steerD_module'

modules = mn_df.motor_module.unique()

for module in modules:
    module_label = str(module)
    seg_ids = mn_df[mn_df.motor_module.isin([module_label])].MN_id.to_list()
    formatted_ids = [str(x) for x in seg_ids]
    
    state = make_json(name = module_label,
                   seg_ids = formatted_ids)
    # new_id = client.state.upload_state_json(state)
    # client.state.build_neuroglancer_url(new_id, ngl_base)

    # save json 
    with open('jsons/{}_MNs.json'.format(module_label), "w") as outfile:
        json.dump(state, outfile)



In [21]:
mn_df.loc[mn_df.index.isin(steerD_mod)]

,MN_id,synapses_include_fragments,total_volume_um3,MN_label,motor_module


In [17]:
pd.read_pickle('wingMN_properties_v604.pkl')

,MN_id,synapses_include_fragments,total_volume_um3,MN_label
0,648518346492244419,11178,4142.678608,PSn_u
1,648518346493634040,9063,5515.440987,b2
2,648518346472269618,8801,8628.326163,DLM_5
3,648518346470491902,8059,5162.048411,i1
4,648518346482131604,7984,3728.941240,iii4a_u
5,648518346471876251,7887,7057.485083,b3_u
6,648518346497125761,7531,3439.709251,PS1
7,648518346480882144,7341,3428.316969,hg4
8,648518346499256923,7248,3980.243544,i2_u
9,648518346486563887,6990,7434.914838,DLM_3


In [7]:
# if you want a link
client.state.build_neuroglancer_url(new_id, ngl_base)

'https://neuromancer-seung-import.appspot.com/?json_url=https://global.daf-apis.com/nglstate/api/v1/5466062256603136'